In [27]:
import pandas as pd
import numpy as np

In [28]:
root = '/home/sihartist/Desktop/'

In order to display all columns

In [29]:
pd.set_option('display.max_columns', None)

In [65]:
original = pd.read_excel(root + 'fraud-detection/dataset/Authorization_data_v2.xlsx', dtype=str)

In [77]:
df = pd.read_excel(root + 'fraud-detection/dataset/final.xlsx', dtype=str)

## Analysis

In [46]:
attr = pd.read_excel(root + 'fraud-detection/dataset/attributes.xlsx', dtype=str)

In [76]:
attr

,PROCESSING_CODE CHAR(2 BYTE),V1
0,FUNCTION_CODE CHAR(3 BYTE),V2
1,NETWORK_CODE CHAR(2 BYTE),V3
2,ISSUING_BANK CHAR(6 BYTE),V4
3,SERVICE_CODE CHAR(3 BYTE),V5
4,START_EXPIRY_DATE DATE,V6
5,END_EXPIRY_DATE DATE,V7
6,TRANSMISSION_DATE_AND_TIME DATE,V8
7,"TRANSACTION_AMOUNT NUMBER(18,3)",V9
8,"BILLING_AMOUNT NUMBER(18,3)",V10
9,"TRANSACTION_FEE NUMBER(18,3)",V11


In [47]:
df.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,CLASS
0,31,281,22,010112,500,01/05/2009,01/05/2011,02/07/2009 21:21:42,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,071181,0,01000002,01000002,26/12/2009,NaN,952,384,VERSUS BANK 2 PLATEAUX REGION LAGUNECI,0
1,31,281,22,010112,500,01/06/2009,01/06/2011,26/06/2009 11:16:47,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,004737,0,01000001,01000001,26/12/2009,NaN,952,384,VERSUS BANK PLATEAU REGION LAGUNECI,0


In [48]:
df.tail(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,CLASS
140652,01,281,22,000010,500,01/01/2009,01/01/2012,02/10/2010 10:30:04,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,016736,0,00000008,01000003,26/12/2010,NaN,952,192,VERSUS BANK ZONE 4 REGION LAGUNECI,1
140653,01,281,22,000010,500,01/07/2009,01/07/2011,27/09/2010 14:53:19,NaN,NaN,NaN,010112,6011,NaN,90,21140121124C,NaN,6,036370,0,01000001,01000001,26/12/2010,NaN,952,360,VERSUS BANK PLATEAU REGION LAGUNECI,1


In [49]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140654 entries, 0 to 140653
Data columns (total 28 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   V1      140654 non-null  object
 1   V2      140654 non-null  object
 2   V3      139696 non-null  object
 3   V4      140645 non-null  object
 4   V5      140654 non-null  object
 5   V6      124864 non-null  object
 6   V7      140654 non-null  object
 7   V8      140654 non-null  object
 8   V9      28907 non-null   object
 9   V10     51372 non-null   object
 10  V11     0 non-null       object
 11  V12     140654 non-null  object
 12  V13     140654 non-null  object
 13  V14     0 non-null       object
 14  V15     11162 non-null   object
 15  V16     140654 non-null  object
 16  V17     0 non-null       object
 17  V18     140616 non-null  object
 18  V19     110978 non-null  object
 19  V20     139679 non-null  object
 20  V21     140654 non-null  object
 21  V22     140654 non-null  object
 

### Preprocessing

Where V1 == 31, put V10 = 0. For non fraud data.    

V2: 300 - 301 (fraude) , 201 (non fraude)       
V3: visa: 001, Mastercard: 002 (both cases)     
V5: remove      
V6: check duration in other instances   
V9: same as billing amount (V10)      
V11: remove     
V14: remove     
V15: duplicate with non fraud values    
V17: remove     
V18: duplicate or remove if not possible    
V19: random 6 numbers   
V20: Origin Code          
V24: same as billing amount        
V12 is Acquirer code ( Aq institution code) 

We can remove the null columns V5, V11, V14, V17, V18 and V20.

In [78]:
df = df.drop(axis=1, columns = ['V5', 'V11', 'V14', 'V17', 'V18', 'V20'])

In [79]:
df_non_fraud = df[df.CLASS == '0']

In [80]:
df_non_fraud.V15.value_counts()

Series([], Name: V15, dtype: int64)

we have the same value everywhere, we will probably remove it later.

In [ ]:
import random
import string

def preprocess_row(df, index):
    if(df['V1'][index] == '31'):
        df.at[index, 'V10'] = 0.0

    # TO DO: Check the duration in other instances and keep it similar. 
    # if(df['V6'][index] is None):
    #     df.at[index, 'V6'] = 
    
    # if(df['V15'][index] is None):
    #     df.at[index, 'V15'] = np.random.choice(V15_values)
    
    if(df['V19'][index] is None):
        df.at[index, 'V15'] = ''.join(random.choices(string.digits, k = 6)) 

    df.at[index, 'V9'] = df['V10'][index]
    df.at[index, 'V24'] = df['V10'][index]

V9, V10 and V24 is the transaction amount. We can cast them into float.

In [81]:
df['V9'] = df['V9'].astype(float)
df['V10'] = df['V10'].astype(float)
df['V24'] = df['V24'].astype(float)
df['CLASS'] = df['CLASS'].astype(int)

#### Analyzing V10 null values

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140654 entries, 0 to 140653
Data columns (total 22 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V1      140654 non-null  object 
 1   V2      140654 non-null  object 
 2   V3      139696 non-null  object 
 3   V4      140645 non-null  object 
 4   V6      124864 non-null  object 
 5   V7      140654 non-null  object 
 6   V8      140654 non-null  object 
 7   V9      28907 non-null   float64
 8   V10     51372 non-null   float64
 9   V12     140654 non-null  object 
 10  V13     140654 non-null  object 
 11  V15     11162 non-null   object 
 12  V16     140654 non-null  object 
 13  V19     110978 non-null  object 
 14  V21     140654 non-null  object 
 15  V22     140654 non-null  object 
 16  V23     140654 non-null  object 
 17  V24     27868 non-null   float64
 18  V25     140654 non-null  object 
 19  V26     140654 non-null  object 
 20  V27     140654 non-null  object 
 21  CLASS   14

We can see that the column V10 have many null values. 
Let's separate the fraud data from the non fraud data to analyse them separately.

In [83]:
n_fraud = df.loc[df['CLASS']==0]
fraud = df.loc[df['CLASS']==1]

In [84]:
n_fraud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70327 entries, 0 to 70326
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      70327 non-null  object 
 1   V2      70327 non-null  object 
 2   V3      70327 non-null  object 
 3   V4      70327 non-null  object 
 4   V6      68351 non-null  object 
 5   V7      70327 non-null  object 
 6   V8      70327 non-null  object 
 7   V9      2827 non-null   float64
 8   V10     2827 non-null   float64
 9   V12     70327 non-null  object 
 10  V13     70327 non-null  object 
 11  V15     0 non-null      object 
 12  V16     70327 non-null  object 
 13  V19     70325 non-null  object 
 14  V21     70327 non-null  object 
 15  V22     70327 non-null  object 
 16  V23     70327 non-null  object 
 17  V24     2827 non-null   float64
 18  V25     70327 non-null  object 
 19  V26     70327 non-null  object 
 20  V27     70327 non-null  object 
 21  CLASS   70327 non-null  int64  
dty

96% of V10 values are null...

In [21]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70327 entries, 70327 to 140653
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      70327 non-null  object 
 1   V4      70327 non-null  object 
 2   V10     52558 non-null  float64
 3   V12     70327 non-null  object 
 4   V13     70327 non-null  object 
 5   V16     70327 non-null  object 
 6   V15     10548 non-null  object 
 7   V21     70327 non-null  object 
 8   V26     70327 non-null  object 
 9   CLASS   70327 non-null  int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 5.9+ MB


25% of V10 are null.    
Let's separate the null values.

In [86]:
v10_nfraud = n_fraud.loc[n_fraud['V10'].isnull()]

In [87]:
v10_nfraud.V1.value_counts()

31    67500
Name: V1, dtype: int64

V1 is always 31. We should fill all the V10 values with 0, but the dataset won't be balanced...

In [89]:
# TO DO
# Fill the nan values of V10

Concatenate with the non NAN values of V10

In [ ]:
v10_non_null = n_fraud.dropna(axis=0, subset=['V10'])

new_n_fraud = pd.concat([v10_non_null, v10_nfraud])
new_n_fraud.info()

### Playing with datetime

In [84]:
df.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,CLASS
0,31,281,22,010112,500,01/05/2009,01/05/2011,02/07/2009 21:21:42,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,071181,0,01000002,01000002,26/12/2009,NaN,952,384,VERSUS BANK 2 PLATEAUX REGION LAGUNECI,0
1,31,281,22,010112,500,01/06/2009,01/06/2011,26/06/2009 11:16:47,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,004737,0,01000001,01000001,26/12/2009,NaN,952,384,VERSUS BANK PLATEAU REGION LAGUNECI,0


In [ ]:
df['V6'] = pd.to_datetime(df['V6'])
df['V7'] = pd.to_datetime(df['V7'])
df['V23'] = pd.to_datetime(df['V23'])

In [88]:
df.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,CLASS
0,31,281,22,010112,500,2009-01-05,2011-01-05,02/07/2009 21:21:42,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,071181,0,01000002,01000002,2009-12-26,NaN,952,384,VERSUS BANK 2 PLATEAUX REGION LAGUNECI,0
1,31,281,22,010112,500,2009-01-06,2011-01-06,26/06/2009 11:16:47,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,004737,0,01000001,01000001,2009-12-26,NaN,952,384,VERSUS BANK PLATEAU REGION LAGUNECI,0


In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140654 entries, 0 to 140653
Data columns (total 28 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   V1      140654 non-null  object        
 1   V2      140654 non-null  object        
 2   V3      139679 non-null  object        
 3   V4      140654 non-null  object        
 4   V5      140654 non-null  object        
 5   V6      124864 non-null  datetime64[ns]
 6   V7      140654 non-null  datetime64[ns]
 7   V8      140654 non-null  object        
 8   V9      28907 non-null   object        
 9   V10     55385 non-null   object        
 10  V11     0 non-null       object        
 11  V12     140654 non-null  object        
 12  V13     140654 non-null  object        
 13  V14     0 non-null       object        
 14  V15     10548 non-null   object        
 15  V16     140654 non-null  object        
 16  V17     0 non-null       object        
 17  V18     140616 non-null  obje

In [95]:
df['year V6'] = df['V6'].dt.year
df.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,CLASS,year V6
0,31,281,22,010112,500,2009-01-05,2011-01-05,02/07/2009 21:21:42,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,071181,0,01000002,01000002,2009-12-26,NaN,952,384,VERSUS BANK 2 PLATEAUX REGION LAGUNECI,0,2009.0
1,31,281,22,010112,500,2009-01-06,2011-01-06,26/06/2009 11:16:47,NaN,NaN,NaN,010112,6011,NaN,NaN,21140121124C,NaN,6,004737,0,01000001,01000001,2009-12-26,NaN,952,384,VERSUS BANK PLATEAU REGION LAGUNECI,0,2009.0
